# データの前処理
* 本ハンズオンでは Personalize の使い方にフォーカスするため、データの前処理を事前に行っている
* データソースからデータをダウンロードして、処理する方法をこのノートブックに記載する

In [ ]:
import pandas as pd

## データの準備
### サンプルデータのダウンロード
* ここではMovielensの10万件の評価データを利用する
* このnotebookではインタラクション、ユーザー、アイテムの3種類のデータを利用する
* まずはデータのダウンロードから
* movielens の ml-100k 10万行のデータセットをダウンロードする

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

## データ前処理方針
* ml-100k の中から 3 種類のデータを作成する。
  * ユーザが映画に対して、どの時系列で、どんな評価を下したのか、というインタラクションデータ
  * 映画を閲覧したユーザがどのような属性（年齢、性別など）なのかをマスタとして持つユーザデータ
  * 各映画がどのジャンルに属するかをマスタとして持つアイテムデータ

### データの加工
#### インタラクションデータの加工
Personalizeのインタラクションスキーマ定義に合わせるため、EVENT_TYPE,EVENT_VALUEカラムを設定  
ここではEVENT_TYPEにRATING、EVENT_VALUEにRATINGの値を設定している  

※Personalizeのレコメンデーションは基本的に特定のイベントの発生可能性が高い順でレコメンデーションのリストを返す形となっており  
　サンプルデータを例とした場合、予想されるRATINGが高い順で結果リストを返すわけではない。  
　例.ユーザーAが映画１を見たか見ていないか？が考慮され、RATINGが高かったどうかはレコメンデーションのリストに反映されない  
　このため、ユーザーが良いRATINGをしそうな映画のみをレコメンデーションのリストに含めたい場合、  
　RATINGが良いレコード（例えば3.5以上）のみを学習データに含める必要がある
  また、ここではEVENT_TYPEを指定しているが、1種類しかイベントの種類が存在しない場合、EVENT_TYPEを使用しないことも可能
 　

In [ ]:
!head -n5 ./ml-100k/u.data

In [ ]:
# データの読み込み
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 10)
data

In [ ]:
# スキーマ定義に合わせたデータ修正
data['EVENT_TYPE'] = 'RATING'
data = data.rename(columns={'RATING': 'EVENT_VALUE'})
data = data.loc[:, ["USER_ID", "ITEM_ID", "EVENT_TYPE", "EVENT_VALUE", "TIMESTAMP"]]
data.head()

In [ ]:
# 3.5以上のRATINGのデータのみに絞る
data = data[data['EVENT_VALUE'] >= 3.5]                

# ここではトレーニング時間を短くするため、利用するデータ量を1万件に絞る場合はコメントアウトを外す
data_sampled = data#.sample(n=10000)
data_sampled.to_csv('data/interaction.csv', index=False)

In [ ]:
!head data/interaction.csv

#### ユーザデータの加工
csv形式で出力するのみ

In [ ]:
!head -n5 ./ml-100k/u.user

In [ ]:
user_data = pd.read_csv('./ml-100k/u.user', sep='|', names=['USER_ID', 'AGE', 'GENDER', 'JOB', 'ZIP'])
user_data.to_csv('data/user.csv', index=False)
user_data.head()

In [ ]:
!head data/user.csv

#### アイテムデータの加工
* ITEM_ID, GENRE の csv 形式にする必要がある
  * GENRE については複数の GENRE を保つ場合があるが（「子供向け」と「コメディ」など）、その場合は GENRE カラム内にパイプ（|）で区切って複数格納する
* u.item ファイルに、アイテムID,アイテム名、属するジャンル の ID が格納されている
* u.genre ファイルに、各ジャンルのIDとジャンル名のマスタデータが格納されている
* 上記 2 ファイルをマージしてアイテムデータを作成する

In [ ]:
!head -n5 ./ml-100k/u.item

In [ ]:
item_data = pd.read_csv('./ml-100k/u.item', sep='|', encoding='latin-1', header=None)
item_data = item_data.drop(3, axis=1)
item_data.head()

In [ ]:
!cat ml-100k/u.genre

In [ ]:
genre_data = pd.read_csv('./ml-100k/u.genre', sep='|', encoding='latin-1', header=None)
genre_data.head()

In [ ]:
column_names = genre_data[0].tolist()
print(column_names)

In [ ]:
column_names.insert(0, 'ITEM_ID')
column_names.insert(1, 'TITLE')
column_names.insert(2, 'RELEASE')
column_names.insert(3, 'IMDB_URL')
print(column_names)

In [ ]:
item_data.columns = column_names
item_data.head()

In [ ]:
def process_genre(item_df):

    genre_list = []
    if(item_df["unknown"] == 1):
        genre_list.append("unknown")
    if(item_df["Action"] == 1):
        genre_list.append("Action")
    if(item_df["Adventure"] == 1):
        genre_list.append("Adventure")
    if(item_df["Animation"] == 1):
        genre_list.append("Animation")
    if(item_df["Children's"] == 1):
        genre_list.append("Children's")
    if(item_df["Comedy"] == 1):
        genre_list.append("Comedy")
    if(item_df["Crime"] == 1):
        genre_list.append("Crime")
    if(item_df["Documentary"] == 1):
        genre_list.append("Documentary")
    if(item_df["Drama"] == 1):
        genre_list.append("Drama")
    if(item_df["Fantasy"] == 1):
        genre_list.append("Fantasy")
    if(item_df["Film-Noir"] == 1):
        genre_list.append("Film-Noir")
    if(item_df["Horror"] == 1):
        genre_list.append("Horror")
    if(item_df["Musical"] == 1):
        genre_list.append("Musical")
    if(item_df["Mystery"] == 1):
        genre_list.append("Mystery")
    if(item_df["Romance"] == 1):
        genre_list.append("Romance")
    if(item_df["Sci-Fi"] == 1):
        genre_list.append("Sci-Fi")
    if(item_df["Thriller"] == 1):
        genre_list.append("Thriller")
    if(item_df["War"] == 1):
        genre_list.append("War")
    if(item_df["Western"] == 1):
        genre_list.append("Western")
        
    return '|'.join(genre_list)



In [ ]:
item_data["GENRE"] = item_data.apply( process_genre, axis=1)

In [ ]:
item_data_csv = item_data[['ITEM_ID', 'GENRE']]
item_data_csv.head()

In [ ]:
item_data_csv.to_csv('data/item.csv', index=False)

In [ ]:
!head data/item.csv